# First Data Analyse

In [1]:
from pandas import read_csv, DataFrame

from numpy import array, zeros

In [3]:
# import the data (take longtime)
# retrive from here https://www.kaggle.com/datasets/stackoverflow/stacksample/data
raw_answers: DataFrame = read_csv('./data/Answers.csv', encoding='latin-1')
raw_questions: DataFrame = read_csv('./data/Questions.csv', encoding='latin-1')
raw_tags: DataFrame = read_csv('./data/Tags.csv', encoding='latin-1')

## Data cleaning and fitting

In [4]:
# creating copy before:
questions = raw_questions.copy()
answers = raw_answers.copy()
tags = raw_tags.copy()

In [5]:
# make a list of tags and link it to questions
tags = tags.groupby('Id')['Tag'].apply(list).reset_index()
questions = questions.join(tags['Tag'], on='Id')
questions

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Tag
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,"[java, windows, swt, executable, software-dist..."
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,"[linux, vmware, sysadmin, nic]"
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql-server, sql-server-2005]"
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,[api]
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[sql, stored-procedures, db2]"
...,...,...,...,...,...,...,...,...
1264211,40143210,5610777.0,2016-10-19T23:38:01Z,NaN,0,URL routing in PHP (MVC),<p>I am building a custom MVC project and I ha...,NaN
1264212,40143300,3791161.0,2016-10-19T23:48:09Z,NaN,0,Bigquery.Jobs.Insert - Resumable Upload?,<p>The API docs show that you should be able t...,NaN
1264213,40143340,7028647.0,2016-10-19T23:52:50Z,NaN,1,Obfuscating code in android studio,<p>Under minifyEnabled I changed from false to...,NaN
1264214,40143360,871677.0,2016-10-19T23:55:24Z,NaN,0,How to fire function after v-model change?,<p>I have input which I use to filter my array...,NaN


In [20]:
# make a user dataset with answer and tags to the attached questions
answers_tmp = answers.join(tags['Tag'], on='ParentId')
most_frequent_users = answers['OwnerUserId'].value_counts()
limit = most_frequent_users.quantile(0.9) # selection 90% best active users for answers
most_frequent_users[most_frequent_users > limit]

OwnerUserId
22656.0      3270
1144035.0    3009
29407.0      2144
548225.0     1924
115145.0     1813
             ... 
240403.0        8
1683466.0       8
526189.0        8
258519.0        8
1460267.0       8
Name: count, Length: 45752, dtype: int64